# Animal Classifier CNN

The following file will integrate a CNN aiming at using the `Animals-10` dataset in order to correctly classify them according to their species. Multiple different hyper parameters, transformations and models will be used through transfer learning to test out different configurations and determine which configuration leads to the best performance.

In order to maintain the code clean, the project will incorporate several helper functions, and will be modularized in the `utils` folder to keep the `main.ipynb` file as clean as possible. 

Throughout the project, the structure can be seen in the outline where after setting up the foundations, the project will focus on tweaking parameters and assessing their performance through tensor board and our own manual prints. 



## 0. Setup

In [9]:
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[0]) >= 2, "torch version should be 2.+"
    assert int(torchvision.__version__.split(".")[1]) >= 15, "torchvision version should be 0.15+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not correct. Installing correct versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

from torch import nn
from torchvision import transforms
import matplotlib as plt

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from tdqm.auto import tqdm
except:
    print(f"[INFO] Couldnt't find tqdm... installing it ")
    !pip install tqdm
    from tqdm.auto import tqdm 


torch version: 2.2.2
torchvision version: 0.17.2
[INFO] Couldnt't find tqdm... installing it 
